# Integración Numérica
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para el curso de <code>Métodos Numéricos</code></p>
<style type="text/css">
    .border {
        display: inline-block;
        border: solid 1px rgba(204, 204, 204, 0.4);
        border-bottom-color: rgba(187, 187, 187, 0.4);
        border-radius: 3px;
        box-shadow: inset 0 -1px 0 rgba(187, 187, 187, 0.4);
        background-color: inherit !important;
        vertical-align: middle;
        color: inherit !important;
        font-size: 11px;
        padding: 3px 5px;
        margin: 0 2px;
    }
</style>

## Solución de integrales por análisis numérico
la integración numérica constituye una amplia gama de algoritmos para calcular el valor numérico de una integral definida

## Agenda
1. Generalidades
1. Método del punto medio
1. Método del trapecio
1. Regla Simpson


In [ ]:
# Importar módulos al cuaderno de jupyter
import math as m 
import numpy as np
import pylab as plt

## 1. Generalidades
---
En un sistema de coordenadas cartesianas, la integral propia de una función $\color{#a78a4d}{f(x)}$ representa el área limitada por la gráfica de la función y el eje de las abscisas $\color{#a78a4d}{x}$ en un intervalo finito, tomando el signo positivo cuando la función toma valores positivos o está definida por encima del eje $\color{#a78a4d}{x}$ y signo negativo cuando la función toma valores negativos o está definida por debajo del eje $\color{#a78a4d}{x}$.

\begin{equation*}
\tag{1} \int_{a}^{b} f(x) \,dx \\
\end{equation*}

Teniendo en cuenta que el problema básico considerado por la integración numérica es calcular una solución aproximada a la integral definida; se tiene que el problema se reduce a calcular el área bajo la curva de la función en un intervalo definido.

<p align="center">
  <img width="400" src="img/definite_integral.png">
</p>

## 2. Métodos del rectangulo
---
En el análisis numérico, los algoritmos del rectángulo son utiles para resolver numéricamente la integral definida; considerando como una aproximaxion del area total, la suma de las areas de $\color{#a78a4d}{n}$ rectangulos.

<p align="center">
  <img width="400" src="img/Riemann_sum.png">
</p>

\begin{align}
\tag{2} A_i &= h_i \cdot b_i \\
\tag{3} A &= \sum h_i \cdot b_i \\
\tag{4} b &= \Delta x \\
\tag{5} A &= \Delta x \sum h_i \\
\end{align}

### a. Suma de Riemann por la izquierda
La suma de Riemann por la izquierda es uno de los algoritmos del rectángulo para resolver numéricamente la integral definida; considerando la altura de cada rectangulo como la función evaluada en el lado izquierdo de ese intervalo.

\begin{align}
\tag{6} A &= \Delta x \sum_{i=0}^{n} f(a + i \cdot \Delta x) \\
\tag{7} \Delta x &= \frac{b-a}{n}  \\
\end{align}

### b. Suma de Riemann por la derecha
La suma de Riemann por la derecha es uno de los algoritmos del rectángulo para resolver numéricamente la integral definida; considerando la altura de cada rectangulo como la función evaluada en el lado derecho de ese intervalo.

\begin{align}
\tag{8} A &= \Delta x \sum_{i=0}^{n} f(b - i \cdot \Delta x) \\
\tag{9} \Delta x &= \frac{b-a}{n}  \\
\end{align}

### c. Método del punto medio
En el análisis numérico, el método del punto medio es uno de los algoritmos del rectángulo para resolver numéricamente la integral definida; considerando la altura de cada rectangulo como la función evaluada en el punto medio de ese intervalo.

\begin{align}
\tag{10} A &= \Delta x \sum_{i=0}^{n} f(a + \frac{2i+1}{2} \cdot \Delta x) \\
\tag{11} \Delta x &= \frac{b-a}{n}  \\
\end{align}

In [ ]:
# Defino los métodos de suma de rectangulos - izquierda
def _Riemman_izq(Func, limA, limB, numI):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (function) : función que depende de una variable.
            limA (function) : derivada de la funcion.
            limB (int)      : semilla de la solución.
            numI (int)      : número máximo de iteraciones.
        
        ## Devoluciones:
            Area (float)       : valor de area encontrado.
    """
    Δx = (limB - limA) / numI
    Alturas = 0
    for i in range(numI):
        Alturas += Func(limA + i * Δx)

    Area = Δx * Alturas
    return Area

# Defino los métodos de suma de rectangulos - izquierda
def _Riemman_der(Func, limA, limB, numI):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (function) : función que depende de una variable.
            limA (function) : derivada de la funcion.
            limB (int)      : semilla de la solución.
            numI (int)      : número máximo de iteraciones.
        
        ## Devoluciones:
            Area (float)       : valor de area encontrado.
    """
    Δx = (limB - limA) / numI
    Alturas = 0
    for i in range(numI):
        Alturas += Func(limB - i * Δx)

    Area = Δx * Alturas
    return Area

# Defino los métodos de suma de rectangulos - izquierda
def _Riemman_med(Func, limA, limB, numI):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (function) : función que depende de una variable.
            limA (function) : derivada de la funcion.
            limB (int)      : semilla de la solución.
            numI (int)      : número máximo de iteraciones.
        
        ## Devoluciones:
            Area (float)       : valor de area encontrado.
    """
    Δx = (limB - limA) / numI
    Alturas = 0
    for i in range(numI):
        Alturas += Func(limA + (2*i + 1)/2 * Δx)

    Area = Δx * Alturas
    return Area

## 3. Método del trapecio
---
En el análisis numérico, el método del trapecio es un algoritmo para resolver numéricamente la integral definida; considerando el area total debajo de la función como la suma de las areas de $\color{#a78a4d}{n}$ trapecios rectos.

\begin{align}
\tag{12} A_i &= \frac{b1_i + b2_i}{2} \cdot h_i \\
\tag{13} A &= \frac{1}{2} \sum h_i \cdot (b1_i + b2_i) \\
\tag{14} h &= \Delta x \\
\tag{15} A &= \frac{ \Delta x }{2} \sum (b1_i + b2_i) \\
\end{align}

In [ ]:
# Defino el métodos del trapecio
def _trapecio(Func, limA, limB, numI):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (function) : función que depende de una variable.
            limA (function) : derivada de la funcion.
            limB (int)      : semilla de la solución.
            numI (int)      : número máximo de iteraciones.
        
        ## Devoluciones:
            Area (float)       : valor de area encontrado.
    """
    Δx = (limB - limA) / numI
    Alturas = 0
    for i in range(numI):
        Alturas += Func(limA + (2*i + 1)/2 * Δx)

    Area = Δx * Alturas
    return Area

## 4. Regla Simpson
---
### a. Ecuaciones lineales
Un caso particular de las ecuaciones algebraicas sucede cuando solo los dos primeros coeficientes son distintos de cero y la solucion para x es unica y trivial.

\begin{align}
a_0 + a_1 x & = 0, \quad a_1 \neq 0 \\
x & = \frac{-a_0}{a_1} \\
\end{align}

### b. Ecuaciones cuadraticas
Un caso particular de las ecuaciones algebraicas sucede cuando solo los tres primeros coeficientes son distintos de cero y las 2 soluciones para x no son compleja.

\begin{align}
a_0 + a_1 x + a_2 x^2 & = 0, \quad a_1 \neq 0 \\
x_1, x_2 & = \frac{-a_1 \pm \sqrt{a_1^2 - 4 a_2 a_0 }}{2 a_2} \\
\end{align}

---
## Mas Recursos

- [Integración numérica](https://es.wikipedia.org/wiki/Integraci%C3%B3n_num%C3%A9rica) (Wikipedia)
- [Suma de Riemann](https://es.wikipedia.org/wiki/Suma_de_Riemann) (Wikipedia)
- [Regla del trapecio](https://es.wikipedia.org/wiki/Regla_del_trapecio) (Wikipedia)
- [Regla de Simpson](https://es.wikipedia.org/wiki/Regla_de_Simpson) (Wikipedia)

